In [1]:
CONFIG_FILE_PATH_PREFIX = 'cfg/'
CONFIG_TMP_FILE_PATH_PREFIX = 'cfg/tmp/'
CONFIG_BACKUP_FILE_PATH_PREFIX = 'cfg/backup/'

OUTPUT_FILE_PATH_PREFIX = 'output/'
OUTPUT_TMP_FILE_PATH_PREFIX = 'output/tmp/'

CONFIG_NUMBER = 1000

In [2]:
#!pip install Faker

In [3]:
import sys
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType,StructField, StringType
spark = SparkSession \
    .builder \
    .appName("Python Spark Customer Fact Generator") \
    .getOrCreate()

22/11/22 10:36:27 WARN Utils: Your hostname, htknphuong-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.142.129 instead (on interface ens33)
22/11/22 10:36:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/22 10:36:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
##Reading the config from file for max id
import os.path
idConfig = {
    "customerrecordid": 10000000,
    "customerid": 10000000,
    "passpost_identitynumber": 111111111,
    "govementid_identitynumber": 111111111,
    "company_identitynumber": 111111111,
    "chiefaccountantid": 10000000,
    "oldercustomersaveid": 10000000,
    "mainaccountid": 10000000,
    "taxcode": 0,
}
print('-----------------')
for file in idConfig.keys():
    file_exist = os.path.exists(CONFIG_FILE_PATH_PREFIX+file)
    if (file_exist):
        print(file + ' is exist')
        print('begin loading' + file)
        realFile = open(CONFIG_FILE_PATH_PREFIX+file,"r")
        idConfig[file] = int(realFile.readlines()[0])
        realFile.close()
    else:
        print(file + 'is not exist')
        print('begin to creating ' + file)
        realFile = open(CONFIG_FILE_PATH_PREFIX+file,"w+")
        realFile.seek(0)
        realFile.write(str(idConfig[file]))
        realFile.close()
    print('-----------------')
    print('begin to creating ' + 'default' + file )
    realFile = open(CONFIG_FILE_PATH_PREFIX+'default'+file,"w+")
    realFile.seek(0)
    realFile.write(str(idConfig[file]))
    realFile.close()

-----------------
customerrecordidis not exist
begin to creating customerrecordid
-----------------
begin to creating defaultcustomerrecordid
customeridis not exist
begin to creating customerid
-----------------
begin to creating defaultcustomerid
passpost_identitynumberis not exist
begin to creating passpost_identitynumber
-----------------
begin to creating defaultpasspost_identitynumber
govementid_identitynumberis not exist
begin to creating govementid_identitynumber
-----------------
begin to creating defaultgovementid_identitynumber
company_identitynumberis not exist
begin to creating company_identitynumber
-----------------
begin to creating defaultcompany_identitynumber
chiefaccountantidis not exist
begin to creating chiefaccountantid
-----------------
begin to creating defaultchiefaccountantid
oldercustomersaveidis not exist
begin to creating oldercustomersaveid
-----------------
begin to creating defaultoldercustomersaveid
mainaccountidis not exist
begin to creating mainaccoun

In [5]:
import random
import faker
from datetime import datetime
import re
class CustomerObject:
    customerType = {
        1: "invidiual",
        2: "company"
    }
    gender = {
        1: "male",
        2: "female"
    }
    personal_identityType = {
        1: "Passpost",
        2: "Govement ID",
    }
    occupationtypetitle = {
        1: "Staff",
        2: "Teamlead",
        3: "Freelancer",
        4: "Deputy Head",
        5: "Deputy",
        6: "Director",
        7: "Vice Director",
        8: "Director General"
    }
    fake = faker.Faker()

    def __init__(self, customerType):
        if(customerType < min(list(CustomerObject.customerType.keys())) or 
           customerType > min(list(CustomerObject.customerType.keys()))):
            __customerType = random.choice(list(CustomerObject.customerType.keys()))
        else:
            __customerType = customerType;
        if (__customerType == 1):
            self.customerrecordid = idConfig["customerrecordid"]
            idConfig["customerrecordid"] += 1
            self.customerid = idConfig["customerid"]
            idConfig["customerid"] += 1
            self.permanentadd = CustomerObject.fake.address()
            self.zipcode = CustomerObject.fake.postcode()
            self.birthday = CustomerObject.fake.date()
            self.firstregisterday = CustomerObject.fake.date_time_between(
                start_date=datetime.strptime(self.birthday, "%Y-%m-%d")).strftime("%Y-%m-%d")
            __gender = random.choice(list(CustomerObject.gender.keys()))
            if (__gender == 1):
                self.name = CustomerObject.fake.name_male()
            elif (__gender == 2):
                self.name = CustomerObject.fake.name_female()
            self.gender = CustomerObject.gender[__gender]
            self.email = CustomerObject.fake.email()
            self.phonenumber = CustomerObject.fake.phone_number()
            self.salary = random.randint(3, 10000000000000)
            self.occupationtypetitle = random.choice(list(CustomerObject.occupationtypetitle.values()))
            self.occupationjobtitle = CustomerObject.fake.job()
            __personal_identitytype = random.choice(list(CustomerObject.personal_identityType.keys()))
            self.identitytype = CustomerObject.personal_identityType[__personal_identitytype] 
            if (__personal_identitytype == 1):
                self.identitynumber = idConfig["passpost_identitynumber"]
                idConfig["passpost_identitynumber"] += 1
            elif (__personal_identitytype == 2):
                self.identitynumber = idConfig["govementid_identitynumber"]
                idConfig["govementid_identitynumber"] += 1
            self.customertype = CustomerObject.customerType[__customerType]
            self.taxcode = idConfig["taxcode"]
            idConfig["taxcode"] += 1
            self.mainbusinesstype = ""
            self.companytypetitle = ""
            self.updatedate = CustomerObject.fake.date_time_between(
                start_date=datetime.strptime(self.firstregisterday, "%Y-%m-%d")).strftime("%Y-%m-%d, %H:%M:%S")
            self.newestcustomerinfo = "True"
            self.mainaccountid = ""
            self.representivename = ""
            self.representivepermanentadd = ""
            self.representivezipcode = ""
            self.representivebirthday = ""
            self.representivegender = ""
            self.representiveemail = ""
            self.representivephonenumber = ""
            self.representiveidentitytype = ""
            self.representiveidentitynumber = ""
            self.chiefaccountantname = ""
            self.chiefaccountantpermanentadd = ""
            self.chiefaccountantzipcode = ""
            self.chiefaccountantbirthday = ""
            self.chiefaccountantgender = ""
            self.chiefaccountantemail = ""
            self.chiefaccountantphonenumber = ""
            self.chiefaccountantidentitytype = ""
            self.chiefaccountantidentitynumber = ""
        elif (__customerType == 2):
            self.customerrecordid = idConfig["customerrecordid"]
            idConfig["customerrecordid"] += 1
            self.customerid = idConfig["customerid"]
            idConfig["customerid"] += 1
            self.name = CustomerObject.fake.company()
            self.permanentadd = CustomerObject.fake.address()
            self.zipcode = CustomerObject.fake.postcode()
            self.birthday = CustomerObject.fake.date()
            self.firstregisterday = CustomerObject.fake.date_time_between(
                start_date=datetime.strptime(self.birthday, "%Y-%m-%d")).strftime("%Y-%m-%d")
            self.gender = ""
            self.email = CustomerObject.fake.email()
            self.phonenumber = CustomerObject.fake.phone_number()
            self.salary = 0
            self.occupationtypetitle = ""
            self.occupationjobtitle = ""
            self.identitytype = "Company Identity"
            self.identitynumber = idConfig["company_identitynumber"]
            idConfig["company_identitynumber"] += 1
            self.customertype = CustomerObject.customerType[__customerType]
            self.taxcode = idConfig["taxcode"]
            idConfig["taxcode"] += 1 
            ## todo
            self.mainbusinesstype = ""
            self.companytypetitle = CustomerObject.fake.company_suffix()
            self.updatedate = CustomerObject.fake.date_time_between(
                start_date=datetime.strptime(self.firstregisterday, "%Y-%m-%d")).strftime("%Y-%m-%d, %H:%M:%S") 
            self.mainaccountid = ""
            __genderrepresentive = random.choice(list(CustomerObject.gender.keys()))
            if (__genderrepresentive == 1):
                self.representivename = CustomerObject.fake.name_male()
            elif (__genderrepresentive == 2):
                self.representivename = CustomerObject.fake.name_female()
            self.representivepermanentadd = CustomerObject.fake.address()
            self.representivezipcode = CustomerObject.fake.postcode()
            self.representivebirthday = CustomerObject.fake.date()
            self.representivegender = __genderrepresentive
            self.representiveemail = CustomerObject.fake.email()
            self.representivephonenumber = CustomerObject.fake.phone_number()
            __personal_representiveidentitytype = random.choice(list(CustomerObject.personal_identityType.keys()))
            self.representiveidentitytype = CustomerObject.personal_identityType[__personal_representiveidentitytype] 
            if (__personal_representiveidentitytype == 1):
                self.representiveidentitynumber = idConfig["passpost_identitynumber"]
                idConfig["passpost_identitynumber"] += 1
            elif (__personal_representiveidentitytype == 2):
                self.representiveidentitynumber = idConfig["govementid_identitynumber"]
                idConfig["govementid_identitynumber"] += 1
            __genderchiefaccountant = random.choice(list(CustomerObject.gender.keys()))
            if (__genderchiefaccountant == 1):
                self.chiefaccountantname = CustomerObject.fake.name_male()
            elif (__genderchiefaccountant == 2):
                self.chiefaccountantname = CustomerObject.fake.name_female()
            self.chiefaccountantpermanentadd = CustomerObject.fake.address()
            self.chiefaccountantzipcode = CustomerObject.fake.postcode()
            self.chiefaccountantbirthday = CustomerObject.fake.date()
            self.chiefaccountantgender = __genderchiefaccountant
            self.chiefaccountantemail = CustomerObject.fake.email()
            self.chiefaccountantphonenumber = CustomerObject.fake.phone_number()
            __personal_chiefaccountantidentitytype = random.choice(list(CustomerObject.personal_identityType.keys()))
            self.chiefaccountantidentitytype = CustomerObject.personal_identityType[__personal_chiefaccountantidentitytype] 
            if (__personal_chiefaccountantidentitytype == 1):
                self.chiefaccountantidentitynumber = idConfig["passpost_identitynumber"]
                idConfig["passpost_identitynumber"] += 1
            elif (__personal_chiefaccountantidentitytype == 2):
                self.chiefaccountantidentitynumber = idConfig["govementid_identitynumber"]
                idConfig["govementid_identitynumber"] += 1
    def toList(self):
        return [Row(
            self.customerrecordid,
            self.customerid,
            self.name,
            self.permanentadd,
            self.zipcode,
            self.birthday,
            self.gender,
            self.email,
            self.phonenumber,
            self.salary,
            self.occupationtypetitle,
            self.occupationjobtitle,
            self.identitytype,
            self.identitynumber,
            self.customertype,
            self.taxcode,
            self.mainbusinesstype,
            self.companytypetitle,
            self.updatedate,
            self.mainaccountid,
            self.representivename,
            self.representivepermanentadd,
            self.representivezipcode,
            self.representivebirthday,
            self.representivegender,
            self.representiveemail,
            self.representivephonenumber,
            self.representiveidentitytype,
            self.representiveidentitynumber,
            self.chiefaccountantname,
            self.chiefaccountantpermanentadd,
            self.chiefaccountantzipcode,
            self.chiefaccountantbirthday,
            self.chiefaccountantgender,
            self.chiefaccountantemail,
            self.chiefaccountantphonenumber,
            self.chiefaccountantidentitytype,
            self.chiefaccountantidentitynumber
            )]


In [6]:
FactCustomer = []
for x in range(CONFIG_NUMBER):
    Customer = CustomerObject(0).toList()
    FactCustomer = FactCustomer + Customer

In [7]:
FactCustomerColumns = StructType([
    StructField("customerrecordid", StringType(), True),
    StructField("customerid", StringType(), True),
    StructField("name", StringType(), True),
    StructField("permanentadd", StringType(), True),
    StructField("zipcode", StringType(), True),
    StructField("birthday", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phonenumber", StringType(), True),
    StructField("salary", StringType(), True),
    StructField("occupationtypetitle", StringType(), True),
    StructField("occupationjobtitle", StringType(), True),
    StructField("identitytype", StringType(), True),
    StructField("identitynumber", StringType(), True),
    StructField("customertype", StringType(), True),
    StructField("taxcode", StringType(), True),
    StructField("mainbusinesstype", StringType(), True),
    StructField("companytypetitle", StringType(), True),
    StructField("updatedate", StringType(), True),
    StructField("mainaccountid", StringType(), True),
    StructField("representivename", StringType(), True),
    StructField("representivepermanentadd", StringType(), True),
    StructField("representivezipcode", StringType(), True),
    StructField("representivebirthday", StringType(), True),
    StructField("representivegender", StringType(), True),
    StructField("representiveemail", StringType(), True),
    StructField("representivephonenumber", StringType(), True),
    StructField("representiveidentitytype", StringType(), True),
    StructField("representiveidentitynumber", StringType(), True),
    StructField("chiefaccountantname", StringType(), True),
    StructField("chiefaccountantpermanentadd", StringType(), True),
    StructField("chiefaccountantzipcode", StringType(), True),
    StructField("chiefaccountantbirthday", StringType(), True),
    StructField("chiefaccountantgender", StringType(), True),
    StructField("chiefaccountantemail", StringType(), True),
    StructField("chiefaccountantphonenumber", StringType(), True),
    StructField("chiefaccountantidentitytype", StringType(), True),
    StructField("chiefaccountantidentitynumber", StringType(), True)
    ])

In [8]:
MainDf = spark.createDataFrame(data=FactCustomer, schema = FactCustomerColumns)

In [9]:
from datetime import datetime
now = datetime.now()
fileName = 'FactCustomer ' + now.strftime("%Y-%m-%d %H-%M-%S-%f") 

In [10]:
print(fileName)

FactCustomer 2022-11-22 10-36-31-502257


In [11]:
MainDf.write.mode('overwrite').option("header","true").csv(OUTPUT_TMP_FILE_PATH_PREFIX + fileName)

22/11/22 10:36:31 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
for file in idConfig.keys():
    print('begin to rewrite ' + file)
    realFile = open(CONFIG_TMP_FILE_PATH_PREFIX+file,"w+")
    realFile.seek(0)
    realFile.write(str(idConfig[file]))
    realFile.close()
    print('-----------------')

begin to rewrite customerrecordid
-----------------
begin to rewrite customerid
-----------------
begin to rewrite passpost_identitynumber
-----------------
begin to rewrite govementid_identitynumber
-----------------
begin to rewrite company_identitynumber
-----------------
begin to rewrite chiefaccountantid
-----------------
begin to rewrite oldercustomersaveid
-----------------
begin to rewrite mainaccountid
-----------------
begin to rewrite taxcode
-----------------


In [13]:
import shutil
for file in idConfig.keys():
    shutil.move(CONFIG_FILE_PATH_PREFIX+file, CONFIG_BACKUP_FILE_PATH_PREFIX+file)
    shutil.move(CONFIG_TMP_FILE_PATH_PREFIX+file, CONFIG_FILE_PATH_PREFIX+file)
shutil.move(OUTPUT_TMP_FILE_PATH_PREFIX + fileName, OUTPUT_FILE_PATH_PREFIX+fileName)

'output/FactCustomer 2022-11-22 10-36-31-502257'